Someone asked how to generate outputs to use with  [LibFFM](https://github.com/guestwalk/libffm)

So all I do is to use pandas cuts for the numerics to turn them into categories.  Feel free to try using them as straight numerics if you wish.  I have tried to make it as generic as possible so you can use it on other competitons going forward!



In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
test.insert(1,'target',0)
print(train.shape)
print(test.shape)

(595212, 59)
(892816, 59)


In [3]:
x = pd.concat([train,test])
x = x.reset_index(drop=True)
unwanted = x.columns[x.columns.str.startswith('ps_calc_')]
x.drop(unwanted,inplace=True,axis=1)

In [4]:
features = x.columns[2:]
categories = []
for c in features:
    trainno = len(x.loc[:train.shape[0],c].unique())
    testno = len(x.loc[train.shape[0]:,c].unique())
    print(c,trainno,testno)

ps_ind_01 8 8
ps_ind_02_cat 5 5
ps_ind_03 12 12
ps_ind_04_cat 3 3
ps_ind_05_cat 8 8
ps_ind_06_bin 2 2
ps_ind_07_bin 2 2
ps_ind_08_bin 2 2
ps_ind_09_bin 2 2
ps_ind_10_bin 2 2
ps_ind_11_bin 2 2
ps_ind_12_bin 2 2
ps_ind_13_bin 2 2
ps_ind_14 5 5
ps_ind_15 14 14
ps_ind_16_bin 2 2
ps_ind_17_bin 2 2
ps_ind_18_bin 2 2
ps_reg_01 10 10
ps_reg_02 19 19
ps_reg_03 5013 5046
ps_car_01_cat 13 13
ps_car_02_cat 3 3
ps_car_03_cat 3 3
ps_car_04_cat 10 10
ps_car_05_cat 3 3
ps_car_06_cat 18 18
ps_car_07_cat 3 3
ps_car_08_cat 2 2
ps_car_09_cat 6 6
ps_car_10_cat 3 3
ps_car_11_cat 104 104
ps_car_11 5 5
ps_car_12 184 201
ps_car_13 70482 83769
ps_car_14 850 885
ps_car_15 15 15


OK so let us turn the big boys into categories

In [5]:
x.loc[:,'ps_reg_03'] = pd.cut(x['ps_reg_03'], 50,labels=False)
x.loc[:,'ps_car_12'] = pd.cut(x['ps_car_12'], 50,labels=False)
x.loc[:,'ps_car_13'] = pd.cut(x['ps_car_13'], 50,labels=False)
x.loc[:,'ps_car_14'] =  pd.cut(x['ps_car_14'], 50,labels=False)
x.loc[:,'ps_car_15'] =  pd.cut(x['ps_car_15'], 50,labels=False)

In [6]:
test = x.loc[train.shape[0]:].copy()
train = x.loc[:train.shape[0]].copy()

In [7]:
#Always good to shuffle for SGD type optimizers
train = train.sample(frac=1).reset_index(drop=True)

In [8]:
train.drop('id',inplace=True,axis=1)
test.drop('id',inplace=True,axis=1)

In [9]:
train.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,0,0,1,4,0,0,1,0,0,0,...,1,1,2,1,32,1,29,2,41,0
1,0,2,1,7,0,0,1,0,0,0,...,1,0,1,1,51,2,30,9,43,49
2,0,7,1,10,0,0,0,1,0,0,...,1,0,0,1,104,2,31,15,44,49
3,0,4,1,5,1,0,0,0,1,0,...,1,1,2,1,68,3,30,9,41,44
4,0,1,1,3,0,0,0,1,0,0,...,1,0,0,1,5,2,31,12,42,48


In [10]:
test.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
595212,0,0,1,8,1,0,0,1,0,0,...,1,1,2,1,65,1,29,5,41,46
595213,0,4,2,5,1,0,0,0,0,1,...,1,1,0,1,103,1,29,4,41,37
595214,0,5,1,3,0,0,0,0,0,1,...,1,1,2,1,29,3,30,8,42,44
595215,0,0,1,6,0,0,1,0,0,0,...,1,1,2,1,40,2,30,5,42,32
595216,0,5,1,7,0,0,0,0,0,1,...,1,1,2,1,101,3,30,7,42,44


All out parameters are categories - if you want to try numerics please scale them first!

In [11]:
categories = train.columns[1:]
numerics = []

In [12]:
currentcode = len(numerics)
catdict = {}
catcodes = {}
for x in numerics:
    catdict[x] = 0
for x in categories:
    catdict[x] = 1

noofrows = train.shape[0]
noofcolumns = len(features)
with open("alltrainffm.txt", "w") as text_file:
    for n, r in enumerate(range(noofrows)):
        if((n%100000)==0):
            print('Row',n)
        datastring = ""
        datarow = train.iloc[r].to_dict()
        datastring += str(int(datarow['target']))


        for i, x in enumerate(catdict.keys()):
            if(catdict[x]==0):
                datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
            else:
                if(x not in catcodes):
                    catcodes[x] = {}
                    currentcode +=1
                    catcodes[x][datarow[x]] = currentcode
                elif(datarow[x] not in catcodes[x]):
                    currentcode +=1
                    catcodes[x][datarow[x]] = currentcode

                code = catcodes[x][datarow[x]]
                datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"
        datastring += '\n'
        text_file.write(datastring)
        
noofrows = test.shape[0]
noofcolumns = len(features)
with open("alltestffm.txt", "w") as text_file:
    for n, r in enumerate(range(noofrows)):
        if((n%100000)==0):
            print('Row',n)
        datastring = ""
        datarow = test.iloc[r].to_dict()
        datastring += str(int(datarow['target']))


        for i, x in enumerate(catdict.keys()):
            if(catdict[x]==0):
                datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
            else:
                if(x not in catcodes):
                    catcodes[x] = {}
                    currentcode +=1
                    catcodes[x][datarow[x]] = currentcode
                elif(datarow[x] not in catcodes[x]):
                    currentcode +=1
                    catcodes[x][datarow[x]] = currentcode

                code = catcodes[x][datarow[x]]
                datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"
        datastring += '\n'
        text_file.write(datastring)


Row 0
Row 100000
Row 200000
Row 300000
Row 400000
Row 500000
Row 0
Row 100000
Row 200000
Row 300000
Row 400000
Row 500000
Row 600000
Row 700000
Row 800000


Once you have built the libffm just use
* ./ffm-train alltrainffm.txt
* ./ffm-predict alltestffm.txt alltrainffm.txt.model output.txt

We haven't shuffled test to we can just create the submission as follows

In [13]:
# sub = pd.read_csv('../input/sample_submission.csv')
# outputs = pd.read_csv('output.txt',header=None)
# outputs.columns = ['target']
# sub.target = outputs.target.ravel()
# sub.to_csv('libffmsubmission.csv',index=False)